In [1]:
import os
%pwd

'd:\\Machine_Learning\\Titanic_Pipeline_Project\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\Machine_Learning\\Titanic_Pipeline_Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    preprocessor_path: Path
    test_data_path: Path
    train_data_path: Path

In [5]:
from titanic.constants import *
from titanic.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            preprocessor_path= config.preprocessor_path,
            test_data_path= config.test_data_path,
            train_data_path= config.train_data_path
        )
        return data_transformation_config

In [15]:
import pandas as pd
import numpy as np
# from titanic import logging
from titanic.logging import logger


from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from titanic.utils.common import save_object


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def load_dataframe(self):
        df = pd.read_csv(os.path.join(self.config.data_path, "Titanic-Dataset.csv"))
        logger.info(f"Loaded {df.shape[0]} rows of data")
        return df
    
    def create_preprocessor(self):

        num_col = ['Age']
        cat_col = ['Sex','Embarked']

        num_pipe = Pipeline(
            steps=[
                # Handling the missing values
                ('imputing_AGE',SimpleImputer(strategy='mean')),
                # scaling 
                ('scalling_data', StandardScaler(with_mean=False)),
            ]
        )

        cat_pipe = Pipeline(
            steps=[
                # Handling the missing values
                ('imputing_AGE',SimpleImputer(strategy='most_frequent')),
                # encoding
                ('one_hot_encoder',OneHotEncoder(handle_unknown='ignore')),
                # scaling
                ('scalling_data', StandardScaler(with_mean=False)),
            ]
        )

        preprocessor = ColumnTransformer(
            [
                ('num_pipeline', num_pipe ,num_col),
                ('cat_pipeline', cat_pipe ,cat_col),
            ]
        )
        logger.info("Preprocessing pipeline")
        return preprocessor


    def save_preprocessor(self, preprocessor_obj):
        save_object(path = Path(self.config.preprocessor_path),obj =preprocessor_obj)
        logger.info("Preprocessor Saved")

    def save_data(self, train_set: pd.DataFrame, test_set:pd.DataFrame):
        train_set.to_csv(self.config.train_data_path, index=False, header=True)
        test_set.to_csv(self.config.test_data_path, index=False, header=True)
        # save_object(path = Path(self.config.train_data_path),obj =train_set)
        save_object(path = Path(self.config.test_data_path),obj =test_set)
        logger.info("Transformed Data Saved")

        
    def initiate_pp(self):
        preprocessor_obj = self.create_preprocessor()
        df = self.load_dataframe()
        
        x = df.drop('Survived', axis = 1)
        y = df['Survived']
        
        # Train test split
        train_set , test_set = train_test_split(df, random_state=42, test_size=0.2)
        self.save_data(train_set, test_set)

        # Preprocessing
        x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=42, test_size=0.2)
        x_train = preprocessor_obj.fit_transform(x_train)
        x_test = preprocessor_obj.transform(x_test)
        self.save_preprocessor(preprocessor_obj)
        
        
        # x_train['Survived'] = y_train
        # x_test['Survived'] = y_test

        
    
    def save_transformed_data(self):
        df = self.initiate_pp()
        

In [16]:
try:
    config = ConfigurationManager()
    data_tranformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_tranformation_config)
    data_transformation.initiate_pp()
    
except Exception as e:
    raise e

[2024-01-08 21:58:59,088: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-01-08 21:58:59,090: INFO: common: yaml file params.yaml loaded successfully]
[2024-01-08 21:58:59,093: INFO: common: created directory at: artifacts]
[2024-01-08 21:58:59,095: INFO: common: created directory at: artifacts/data_transformation]
[2024-01-08 21:58:59,096: INFO: 3692943396: Preprocessing pipeline]
[2024-01-08 21:58:59,102: INFO: 3692943396: Loaded 891 rows of data]
[2024-01-08 21:58:59,114: INFO: 3692943396: Transformed Data Saved]
[2024-01-08 21:58:59,134: INFO: 3692943396: Preprocessor Saved]
